# Streamlit Deployment Prep
In this notebook we will be creating more tailored models for each of our four cities. We will be using a [free weather api](https://www.weatherapi.com/) that will let us get current weather data for any gps coordinates that we choose and we will be translating this to weather data that our model can use.

In [1]:
import requests
import pandas as pd
import numpy as np
import pickle
import datetime as dt
from dateutil import parser
import folium
from branca.element import Figure

import xgboost as xg
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier
from sklearn.metrics import plot_confusion_matrix, classification_report

In [2]:
pd.set_option("display.max_columns", None)
chi_data = pd.read_csv('../data/chi_df_cleaned.csv')
chi_data.head()

,severity,start_lat,start_lng,end_lat,end_lng,distance(mi),temperature(f),wind_chill(f),humidity(%),pressure(in),visibility(mi),wind_speed(mph),precipitation(in),amenity,bump,crossing,give_way,junction,no_exit,railway,roundabout,station,stop,traffic_calming,traffic_signal,turning_loop,year,month,week,start_time_ep,end_time_ep,weather_timestamp_ep,accident_duration,side_L,side_R,wind_direction_CALM,wind_direction_E,wind_direction_ENE,wind_direction_ESE,wind_direction_N,wind_direction_NE,wind_direction_NNE,wind_direction_NNW,wind_direction_NW,wind_direction_S,wind_direction_SE,wind_direction_SSE,wind_direction_SSW,wind_direction_SW,wind_direction_VAR,wind_direction_W,wind_direction_WNW,wind_direction_WSW,weather_condition_Blowing Snow,weather_condition_Clear,weather_condition_Cloudy,weather_condition_Cloudy / Windy,weather_condition_Drizzle,weather_condition_Fair,weather_condition_Fair / Windy,weather_condition_Fog,weather_condition_Freezing Rain,weather_condition_Haze,weather_condition_Heavy Drizzle,weather_condition_Heavy Rain,weather_condition_Heavy Rain / Windy,weather_condition_Heavy Snow,weather_condition_Heavy T-Storm,weather_condition_Heavy T-Storm / Windy,weather_condition_Heavy Thunderstorms and Rain,weather_condition_Light Drizzle,weather_condition_Light Drizzle / Windy,weather_condition_Light Freezing Drizzle,weather_condition_Light Freezing Fog,weather_condition_Light Freezing Rain,weather_condition_Light Freezing Rain / Windy,weather_condition_Light Ice Pellets,weather_condition_Light Rain,weather_condition_Light Rain / Windy,weather_condition_Light Rain with Thunder,weather_condition_Light Snow,weather_condition_Light Snow / Windy,weather_condition_Light Snow and Sleet / Windy,weather_condition_Light Thunderstorms and Rain,weather_condition_Light Thunderstorms and Snow,weather_condition_Mist,weather_condition_Mostly Cloudy,weather_condition_Mostly Cloudy / Windy,weather_condition_N/A Precipitation,weather_condition_Overcast,weather_condition_Partly Cloudy,weather_condition_Partly Cloudy / Windy,weather_condition_Rain,weather_condition_Rain / Windy,weather_condition_Scattered Clouds,weather_condition_Smoke,weather_condition_Snow,weather_condition_Snow / Windy,weather_condition_Squalls,weather_condition_Squalls / Windy,weather_condition_T-Storm / Windy,weather_condition_Thunder,weather_condition_Thunder in the Vicinity,weather_condition_Thunderstorm,weather_condition_Thunderstorms and Rain,weather_condition_Wintry Mix,weather_condition_Wintry Mix / Windy,sunrise_sunset_Day,sunrise_sunset_Night,civil_twilight_Day,civil_twilight_Night,nautical_twilight_Day,nautical_twilight_Night,astronomical_twilight_Day,astronomical_twilight_Night,day_Friday,day_Monday,day_Saturday,day_Sunday,day_Thursday,day_Tuesday,day_Wednesday,hour_00,hour_01,hour_02,hour_03,hour_04,hour_05,hour_06,hour_07,hour_08,hour_09,hour_10,hour_11,hour_12,hour_13,hour_14,hour_15,hour_16,hour_17,hour_18,hour_19,hour_20,hour_21,hour_22,hour_23,is_DST
0,2,42.305960,-87.960150,42.306916,-87.960918,0.077,37.0,31.0,79.0,29.59,10.0,8.1,0.0,0,0,0,0,0,0,0,0,0,0,0,1,0,2016,11,48,1.480517e+09,1.480539e+09,1.480518e+09,21600.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
1,2,42.175870,-88.135770,42.174620,-88.135794,0.086,37.0,28.1,76.0,29.62,10.0,15.0,0.0,0,0,0,0,0,0,0,0,0,0,0,1,0,2016,11,48,1.480518e+09,1.480540e+09,1.480518e+09,21600.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
2,3,42.175897,-88.135769,42.174612,-88.135794,0.089,37.0,28.1,76.0,29.62,10.0,15.0,0.0,0,0,0,0,0,0,0,0,0,0,0,1,0,2016,11,48,1.480519e+09,1.480541e+09,1.480518e+09,22524.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,

There are several weather columns that will not align with the data we will get in our api calls so we'll need to do a bit of condensing and reassigning. Since there is only ever one weather condition we can just take the sum of all the columns we want to combine and drop out the extra columns.

In [3]:
chi_data['weather_condition_Clear'] = chi_data['weather_condition_Clear'] + chi_data['weather_condition_Fair'] + chi_data['weather_condition_Fair / Windy'] + chi_data['weather_condition_N/A Precipitation']
chi_data['weather_condition_Cloudy'] = chi_data['weather_condition_Cloudy'] + chi_data['weather_condition_Cloudy / Windy'] + chi_data['weather_condition_Mostly Cloudy'] + chi_data['weather_condition_Mostly Cloudy / Windy']
chi_data['weather_condition_Fog'] = chi_data['weather_condition_Fog'] + chi_data['weather_condition_Haze'] + chi_data['weather_condition_Smoke']
chi_data['weather_condition_Heavy Rain'] = chi_data['weather_condition_Heavy Rain'] + chi_data['weather_condition_Heavy Rain / Windy'] + chi_data['weather_condition_Heavy Drizzle'] + chi_data['weather_condition_Squalls'] + chi_data['weather_condition_Squalls / Windy']
chi_data['weather_condition_Light Drizzle'] = chi_data['weather_condition_Light Drizzle'] + chi_data['weather_condition_Light Drizzle / Windy']
chi_data['weather_condition_Light Freezing Rain'] = chi_data['weather_condition_Light Freezing Rain'] + chi_data['weather_condition_Light Freezing Rain / Windy']
chi_data['weather_condition_Light Rain'] = chi_data['weather_condition_Light Rain'] + chi_data['weather_condition_Light Rain / Windy']
chi_data['weather_condition_Light Snow'] = chi_data['weather_condition_Light Snow'] + chi_data['weather_condition_Light Snow / Windy']
chi_data['weather_condition_Light Snow and Sleet / Windy'] = chi_data['weather_condition_Light Snow and Sleet / Windy'] + chi_data['weather_condition_Wintry Mix'] + chi_data['weather_condition_Wintry Mix / Windy']
chi_data['weather_condition_Partly Cloudy'] = chi_data['weather_condition_Partly Cloudy'] + chi_data['weather_condition_Partly Cloudy / Windy'] + chi_data['weather_condition_Scattered Clouds']
chi_data['weather_condition_Rain'] = chi_data['weather_condition_Rain'] + chi_data['weather_condition_Rain / Windy'] + chi_data['weather_condition_Drizzle']
chi_data['weather_condition_Snow'] = chi_data['weather_condition_Snow'] + chi_data['weather_condition_Snow / Windy']
chi_data['weather_condition_Thunderstorms and Rain'] = chi_data['weather_condition_Thunderstorms and Rain'] + chi_data['weather_condition_Thunderstorm'] + chi_data['weather_condition_T-Storm / Windy'] + chi_data['weather_condition_Heavy T-Storm'] + chi_data['weather_condition_Heavy T-Storm / Windy'] + chi_data['weather_condition_Heavy Thunderstorms and Rain']
chi_data['weather_condition_Light Rain with Thunder'] = chi_data['weather_condition_Light Rain with Thunder'] + chi_data['weather_condition_Light Thunderstorms and Rain']
chi_data['weather_condition_Thunder'] = chi_data['weather_condition_Thunder'] + chi_data['weather_condition_Thunder in the Vicinity']

weather_drop = ['weather_condition_Fair','weather_condition_Fair / Windy','weather_condition_N/A Precipitation','weather_condition_Cloudy / Windy','weather_condition_Mostly Cloudy','weather_condition_Haze','weather_condition_Smoke',
                'weather_condition_Heavy Rain / Windy','weather_condition_Heavy Drizzle','weather_condition_Squalls','weather_condition_Squalls / Windy','weather_condition_Light Drizzle / Windy','weather_condition_Light Freezing Rain / Windy',
                'weather_condition_Light Rain / Windy','weather_condition_Light Snow / Windy','weather_condition_Wintry Mix','weather_condition_Wintry Mix / Windy','weather_condition_Partly Cloudy / Windy','weather_condition_Scattered Clouds',
                'weather_condition_Rain / Windy','weather_condition_Drizzle','weather_condition_Snow / Windy','weather_condition_Thunderstorm','weather_condition_T-Storm / Windy','weather_condition_Heavy T-Storm','weather_condition_Heavy T-Storm / Windy',
                'weather_condition_Heavy Thunderstorms and Rain','weather_condition_Light Thunderstorms and Rain','weather_condition_Thunder in the Vicinity']

Clear -- Fair -- Fair / Windy -- N/A Precipitation
Cloudy -- Cloudy / Windy -- Mostly Cloudy -- Mostly Cloudy / Windy
Fog -- Patches of Fog -- Haze -- Smoke
Heavy Rain -- Heavy Rain / Windy -- Heavy Drizzle -- Squalls -- Squalls / Windy
Light Drizzle -- Light Drizzle / Windy
Light Freezing Rain -- Light Freezing Rain / Windy
Light Rain -- Light Rain / Windy
Light Snow -- Light Snow / Windy
Light Snow and Sleet / Windy -- Wintry Mix -- Wintry Mix / Windy
Partly Cloudy -- Partly Cloudy / Windy -- Scattered Clouds
Rain -- Drizzle -- Rain / Windy
Snow -- Snow / Windy
Thunderstorms and Rain -- Thunderstorm -- T-Storm -- T-Storm / Windy -- Heavy T-Storm -- Heavy T-Storm / Windy -- Heavy Thunderstorms and Rain
Light Rain with Thunder -- Light Thunderstorms and Rain
Thunder -- Thunder in the Vicinity

In [4]:
drop_features = ['severity','end_lat','end_lng','distance(mi)',
                 'amenity','bump','crossing','give_way','junction','no_exit','railway','roundabout','station','stop','traffic_calming','traffic_signal','turning_loop',
                 'end_time_ep','weather_timestamp_ep','accident_duration','side_R','sunrise_sunset_Day','sunrise_sunset_Night','civil_twilight_Day','civil_twilight_Night',
                 'nautical_twilight_Day','nautical_twilight_Night','astronomical_twilight_Day','astronomical_twilight_Night']

X = chi_data.drop(drop_features+weather_drop, axis=1)
y = chi_data['severity']
# split
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=42)
print('X_train shape:',X_train.shape)
print('y_train shape:',y_train.shape)
print('X_test shape:',X_test.shape)
print('y_test shape:',y_test.shape)

X_train shape: (30730, 89)
y_train shape: (30730,)
X_test shape: (10244, 89)
y_test shape: (10244,)


In [5]:
X_train.head()

,start_lat,start_lng,temperature(f),wind_chill(f),humidity(%),pressure(in),visibility(mi),wind_speed(mph),precipitation(in),year,month,week,start_time_ep,side_L,wind_direction_CALM,wind_direction_E,wind_direction_ENE,wind_direction_ESE,wind_direction_N,wind_direction_NE,wind_direction_NNE,wind_direction_NNW,wind_direction_NW,wind_direction_S,wind_direction_SE,wind_direction_SSE,wind_direction_SSW,wind_direction_SW,wind_direction_VAR,wind_direction_W,wind_direction_WNW,wind_direction_WSW,weather_condition_Blowing Snow,weather_condition_Clear,weather_condition_Cloudy,weather_condition_Fog,weather_condition_Freezing Rain,weather_condition_Heavy Rain,weather_condition_Heavy Snow,weather_condition_Light Drizzle,weather_condition_Light Freezing Drizzle,weather_condition_Light Freezing Fog,weather_condition_Light Freezing Rain,weather_condition_Light Ice Pellets,weather_condition_Light Rain,weather_condition_Light Rain with Thunder,weather_condition_Light Snow,weather_condition_Light Snow and Sleet / Windy,weather_condition_Light Thunderstorms and Snow,weather_condition_Mist,weather_condition_Mostly Cloudy / Windy,weather_condition_Overcast,weather_condition_Partly Cloudy,weather_condition_Rain,weather_condition_Snow,weather_condition_Thunder,weather_condition_Thunderstorms and Rain,day_Friday,day_Monday,day_Saturday,day_Sunday,day_Thursday,day_Tuesday,day_Wednesday,hour_00,hour_01,hour_02,hour_03,hour_04,hour_05,hour_06,hour_07,hour_08,hour_09,hour_10,hour_11,hour_12,hour_13,hour_14,hour_15,hour_16,hour_17,hour_18,hour_19,hour_20,hour_21,hour_22,hour_23,is_DST
781,42.25602,-88.15199,18.0,9.8,88.0,29.85,0.8,5.800000,0.02,2016,12,50,1.481913e+09,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
17045,41.57765,-87.56348,91.0,91.0,38.0,29.17,10.0,10.003125,0.00,2020,6,22,1.591119e+09,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
25455,41.98323,-87.81810,28.0,19.0,47.0,29.52,10.0,9.000000,0.00,2019,11,44,1.573246e+09,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1
29490,41.82914,-88.09650,48.0,48.0,61.0,30.05,10.0,11.500000,0.00,2018,10,41,1.539723e+09,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
19779,41.76321,-87.85790,75.0,75.0,94.0,28.79,1.0,21.000000,0.06,2020,6,23,1.591724e+09,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0


In [6]:
y.value_counts(normalize=True)

3    0.507615
2    0.412481
4    0.073827
1    0.006077
Name: severity, dtype: float64

In [7]:
xg_model = xg.sklearn.XGBClassifier()
xg_model.fit(X_train,y_train)
print('XGBoost Training Accuracy:', xg_model.score(X_train,y_train))
print('XGBoost Testing Accuracy:', xg_model.score(X_test,y_test))

C:\ProgramData\Anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[11:09:36] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.0/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
XGBoost Training Accuracy: 0.8930686625447446
XGBoost Testing Accuracy: 0.8363920343615775


In [8]:
#save the model
with open('./models/chicago.pkl', mode='wb') as pickle_out:
    pickle.dump(xg_model, pickle_out)
with open('./models/chicago_cols.pkl', mode='wb') as pickle_out:
    pickle.dump(X_train.columns, pickle_out)

In [9]:
lat = 41.8781
lon = -87.6298
url = 'http://api.weatherapi.com/v1'
slug = '/current.json'
parameters = {
    'key': '0cb381c6bf3c4d4384b155640212911',
    'q': f'{lat},{lon}'
}

res = requests.get(url+slug, parameters)
weather_data = res.json()

In [26]:
current_time = parser.parse(weather_data['location']['localtime'])
prediction_params = {
    'start_lat': lat,
    'start_lng': lon,
    'temperature(f)': weather_data['current']['temp_f'],
    'wind_chill(f)': weather_data['current']['feelslike_f'],
    'humidity(%)': weather_data['current']['humidity'],
    'pressure(in)': weather_data['current']['pressure_in'],
    'visibility(mi)': weather_data['current']['vis_miles'],
    'wind_speed(mph)': weather_data['current']['wind_mph'],
    'precipitation(in)': weather_data['current']['precip_in'],
    'year': current_time.strftime('%Y'),
    'month': current_time.strftime('%m'),
    'week': current_time.strftime('%U'),
    'start_time_ep': (current_time - dt.datetime(1970,1,1)).total_seconds(),
    f"wind_direction_{weather_data['current']['wind_dir']}": 1,
    f"day_{current_time.strftime('%A')}": 1,
    f"hour_{current_time.strftime('%H')}": 1
    
}
weather_string = f"{weather_data['current']['condition']['text']}\n{weather_data['current']['temp_f']} F ({weather_data['current']['feelslike_f']} F)\nHumidity: {weather_data['current']['humidity']}%\nPressure: {weather_data['current']['pressure_in']}\nVisibility: {weather_data['current']['vis_miles']} mi\nWind Spd/Dir: {weather_data['current']['wind_mph']} mph {weather_data['current']['wind_dir']}\nPrecipitation(in): {weather_data['current']['precip_in']}"

In [11]:
pred_df = pd.DataFrame(columns=X_train.columns)
pred_df = pred_df.append(prediction_params, ignore_index=True).fillna(0)
pred_df

,start_lat,start_lng,temperature(f),wind_chill(f),humidity(%),pressure(in),visibility(mi),wind_speed(mph),precipitation(in),year,month,week,start_time_ep,side_L,wind_direction_CALM,wind_direction_E,wind_direction_ENE,wind_direction_ESE,wind_direction_N,wind_direction_NE,wind_direction_NNE,wind_direction_NNW,wind_direction_NW,wind_direction_S,wind_direction_SE,wind_direction_SSE,wind_direction_SSW,wind_direction_SW,wind_direction_VAR,wind_direction_W,wind_direction_WNW,wind_direction_WSW,weather_condition_Blowing Snow,weather_condition_Clear,weather_condition_Cloudy,weather_condition_Fog,weather_condition_Freezing Rain,weather_condition_Heavy Rain,weather_condition_Heavy Snow,weather_condition_Light Drizzle,weather_condition_Light Freezing Drizzle,weather_condition_Light Freezing Fog,weather_condition_Light Freezing Rain,weather_condition_Light Ice Pellets,weather_condition_Light Rain,weather_condition_Light Rain with Thunder,weather_condition_Light Snow,weather_condition_Light Snow and Sleet / Windy,weather_condition_Light Thunderstorms and Snow,weather_condition_Mist,weather_condition_Mostly Cloudy / Windy,weather_condition_Overcast,weather_condition_Partly Cloudy,weather_condition_Rain,weather_condition_Snow,weather_condition_Thunder,weather_condition_Thunderstorms and Rain,day_Friday,day_Monday,day_Saturday,day_Sunday,day_Thursday,day_Tuesday,day_Wednesday,hour_00,hour_01,hour_02,hour_03,hour_04,hour_05,hour_06,hour_07,hour_08,hour_09,hour_10,hour_11,hour_12,hour_13,hour_14,hour_15,hour_16,hour_17,hour_18,hour_19,hour_20,hour_21,hour_22,hour_23,is_DST
0,41.8781,-87.6298,41.0,35.1,75,29.98,9.0,8.1,0.0,2021,12,48,1.638355e+09,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [12]:
xg_model.predict(pred_df)

array([2], dtype=int64)

In [27]:
fig = Figure(width=550, height=350)
m1 = folium.Map(width=550, height=350, location=[lat,lon], zoom_start=10, max_zoom=12, min_zoom=8)
fig.add_child(m1)
folium.Marker(location=[lat,lon], popup='marker 1').add_to(m1)
folium.Marker(location=[lat,lon], popup=f'{weather_string}', ).add_to(m1)
m1

In [ ]:
{
    'location': {
        'name': 'Chicago',
        'region': 'Illinois',
        'country': 'United States of America',
        'lat': 41.85,
        'lon': -87.65,
        'tz_id': 'America/Chicago',
        'localtime_epoch': 1638216524,
        'localtime': '2021-11-29 14:08'
    },
    'current': {
        'last_updated_epoch': 1638212400,
        'last_updated': '2021-11-29 13:00',
        'temp_c': 5.0,
        'temp_f': 41.0,
        'is_day': 1,
        'condition': {
            'text': 'Overcast',
            'icon': '//cdn.weatherapi.com/weather/64x64/day/122.png',
            'code': 1009
        },
        'wind_mph': 11.9,
        'wind_kph': 19.1,
        'wind_degree': 190,
        'wind_dir': 'S',
        'pressure_mb': 1016.0,
        'pressure_in': 30.0,
        'precip_mm': 0.0,
        'precip_in': 0.0,
        'humidity': 36,
        'cloud': 100,
        'feelslike_c': 1.3,
        'feelslike_f': 34.4,
        'vis_km': 16.0,
        'vis_miles': 9.0,
        'uv': 1.0,
        'gust_mph': 13.9,
        'gust_kph': 22.3
    }
}

start_lat
start_lng
temperature(f)
wind_chill(f)
humidity(%)
pressure(in)
visibility(mi)
wind_speed(mph)
precipitation(in)
year
month
week
start_time_ep

In [ ]:
condition_dict{
    1000 : 'Clear',
    1006 : 'Cloudy',
    1135 : 'Fog',
    1171 : 'Freezing Rain',
    1201 : 'Freezing Rain',
    1195 : 'Heavy Rain',
    1192 : 'Heavy Rain',
    1243 : 'Heavy Rain',
    1246 : 'Heavy Rain',
    1225 : 'Heavy Snow',
    1117 : 'Heavy Snow',
    1114 : 'Heavy Snow',
    1258 : 'Heavy Snow',
    1222 : 'Heavy Snow',
    1153 : 'Light Drizzle',
    1150 : 'Light Drizzle',
    1168 : 'Light Freezing Drizzle',
    1072 : 'Light Freezing Drizzle',
    1147 : 'Light Freezing Fog',
    1198 : 'Light Freezing Rain',
    1261 : 'Light Ice Pellets',
    1237 : 'Light Ice Pellets',
    1264 : 'Light Ice Pellets',
    1153 : 'Light Rain',
    1183 : 'Light Rain',
    1240 : 'Light Rain Showers',
    1180 : 'Light Rain Showers',
    1213 : 'Light Snow',
    1255 : 'Light Snow',
    1210 : 'Light Snow',
    1204 : 'Light Snow and Sleet / Windy',
    1249 : 'Light Snow and Sleet / Windy',
    1207 : 'Light Snow and Sleet / Windy',
    1252 : 'Light Snow and Sleet / Windy',
    1069 : 'Light Snow and Sleet / Windy',
    1030 : 'Mist',
    1009 : 'Overcast',
    1003 : 'Partly Cloudy',
    1189 : 'Rain',
    1186 : 'Rain',
    1063 : 'Rain',
    1219 : 'Snow',
    1216 : 'Snow',
    1066 : 'Snow',
    1276 : 'Thunderstorms and Rain',
    1282 : 'Light Thunderstorms and Snow',
    1279 : 'Light Thunderstorms and Snow',
    1273 : 'Light Rain with Thunder',
    1087 : 'Thunder',
}